In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import bibtexparser
import json

ieeeSearchAPI = "https://ieeexplore.ieee.org/rest/search"
ieeeExportAPI = "https://ieeexplore.ieee.org/xpl/downloadCitations"

## Downloading from IEEE APIs

In [8]:
searchString = "((\"Full Text .AND. Metadata\":\"Ubiquitous Computing\") OR \"Full Text .AND. Metadata\":\"Pervasive Computing\")"

In [33]:
targetQuantity = 1200
currentNumber = 0

dic = {'title':[], 'journal':[], 'year':[], 'keywords':[]}
articleIds = []
pageNum = 1
while currentNumber < targetQuantity:
    #GET ID's
    rAPI = requests.post(ieeeSearchAPI, json={"queryText":searchString, "returnType":"SEARCH", "rowsPerPage":"100", "pageNumber":pageNum}, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0','Origin': 'https://ieeexplore.ieee.org'})
    while(rAPI.status_code != 200):
        rAPI = requests.post(ieeeSearchAPI, json={"queryText":searchString, "returnType":"SEARCH", "rowsPerPage":"100", "pageNumber":pageNum}, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0','Origin': 'https://ieeexplore.ieee.org'})
    
    #PUT IN LIST
    try:
        parsed_json = (json.loads(rAPI.content.decode()))
    except:
        print(pageNum)
        print(rAPI.content)
    articleIds = articleIds + [x["articleNumber"] for x in parsed_json["records"]]
    
    #NEXT PAGE
    pageNum = pageNum + 1
    currentNumber = currentNumber + 100

    
count = 0
while count < targetQuantity:
    # GET BIBTEX
    if(count+100 > targetQuantity-1):
        articleString = ','.join(articleIds[count:targetQuantity])
    else:  
        articleString = ','.join(articleIds[count:count+100])
    count = count + 100
    rAPI = requests.post(ieeeExportAPI, data={"recordIds":articleString, "download-format":"download-bibtex", "citations-format":"citation-only"}, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0','Origin': 'https://ieeexplore.ieee.org'})

    # CLEAN BIBTEX
    dirty_bibdata = rAPI.content.decode()
    dirty_bibdata = dirty_bibdata.replace("<br>@","\n@")
    dirty_bibdata = dirty_bibdata.replace("<br>","")
    #print(dirty_bibdata)
    bibdata = bibtexparser.loads(dirty_bibdata)
    
    # BIBTEXT PARSE
    for bib_id in bibdata.entries:
        try:
            dic['title'].append(bib_id["title"])
        except(KeyError):
            dic['title'].append("")
        try:
            dic['journal'].append(bib_id['booktitle'])
        except(KeyError):
            dic['journal'].append("")
        try:
            dic['year'].append(bib_id['year'])
        except(KeyError):
            dic['year'].append("")    
        try:
            dic['keywords'].append(bib_id['keywords'])
        except(KeyError):
            dic['keywords'].append("")
    
print(len(dic['title']))
print(len(dic['journal']))
print(len(dic['keywords']))
print(len(dic['year']))

# PUT IN CSV
df = pd.DataFrame.from_dict(dic)
df.to_csv("genericIEEE.csv",index=False,sep=",")


1200
1200
1200
1200
